### imdb 是一个用于检索和管理 imdb 电影数据的 python 包，相当于一个封装好的爬虫
- 文档：https://imdbpy.readthedocs.io/en/latest/usage/index.html

### 需要解决的问题如下：

1. 如何指定电影的 id？【已解决】

答：爬取电影列表 https://www.imdb.com/search/title/?title_type=feature&year=1950-01-01,2012-12-31&sort=num_votes,desc&ref_=adv_prv
    
   可以根据 imdb 数据集中第二个特征值是 "movie" 的进行筛选

2. 指定哪些属性字段？【已解决】

答：文档中有说，属性标签是根据 web 网页来的，所以 key 是什么其实不确定，要在抓取前看一下

   已经指定完毕，30个

### 临时思考记录
- 用电影数据（类型年份等属性）构建的是属性三元组
- 但我们要构建的是关系三元组 （电影-人物关系、电影-公司关系），三元组的尾实体也是一个有独立页面的主体，而不是无意义的年份或电影类型

#### 根据指定的电影 id 返回电影信息

In [ ]:
import imdb

ia = imdb.IMDb()
movie = ia.get_movie('0120382')

print(movie)

In [ ]:
print(movie["directors"])
print(movie["genres"])

for director in movie["directors"]:
    print(director["name"])

for genre in movie["genres"]:
    print(genre)

#### 批量电影信息

In [ ]:
top250 = ia.get_top250_movies()
bottom100 = ia.get_bottom100_movies()
popular100 = ia.get_popular100_movies()
top250_tv = ia.get_top250_tv()
top250_indian_movie = ia.get_top250_indian_movies()

In [ ]:
print(top250[0])

#### 属性字段

In [ ]:
infoset = ia.get_movie_infoset()
print(infoset)

In [ ]:
print(movie.infoset2keys)

### 有三种实体：
- 电影实体：The Truman Show    tt0120382
- 人物实体：Jim Carrey         nm0000120
- 公司实体：Paramount Pictures      co0023400

### 关系有30个：
- 卡司 movie["cast"]
- 导演 movie["directors"]
- 编剧 movie["writers"]
- 制片 movie["producers"]
- 作曲 movie["composers"]
- 摄影 movie["cinematographers"]
- 剪辑 movie["editors"]
- 剪辑部门 movie["editorial department"]
- 选角导演 movie["casting directors"]
- 制作设计 movie["production designers"]

- 艺术总监 movie["art directors"]
- 布景装饰 movie["set decorators"]
- 服装设计 movie["costume designers"]
- 化妆部门 movie["make up department"]
- 制作经理 movie["production managers"]
- 助理导演 movie["assistant directors"]
- 艺术部门 movie["art department"]
- 声音部门 movie["sound department"]
- 特效     movie["special effects"]
- 视觉效果 movie["visual effects"]

- 特技     movie["stunts"]
- 摄影部门 movie["camera department"]
- 选角部门 movie["casting department"]
- 服装部门 movie["costume departmen"]
- 位置管理 movie["location management"]
- 音乐部门 movie["music department"]
- 剧本部门 movie["script department"]
- 运输部门 movie["transportation department"]
- 制片公司 movie["production companies"]    【公司】
- 发行商 movie["distributors"]              【公司】

In [ ]:
for attri in movie.infoset2keys['main']:
    print(attri + " : " + str(movie[attri]) + "\n")

In [ ]:
for attri in movie.infoset2keys['plot']:
    print(attri + " : " + str(movie[attri]))

In [ ]:
# help(imdb.Person.Person)
# help(imdb.Company.Company)

print(movie["writers"][0].personID)    # 获取人物 ID，str 类型
print(movie["writers"][0]['name'])     # 获取人物姓名，str 类型

print(movie["production companies"][0].companyID)    # 获取公司 ID，str 类型
print(movie["production companies"][0]['name'])      # 获取公司名字，str 类型

---

### IMDB 公开了数据集：https://www.imdb.com/interfaces/
（想做某件事情时候先看别人做了什么工作，做到什么程度，在前人的基础上去做，避免重复造轮子）

### 构建三元组知识图谱的步骤：
0. 指定属性标签【完成】
1. 从 imdb 数据集中用 pandas 筛选出所有第二维度属性值为 movie 的条目的 tt 值【完成】
2. 根据 tt 值调用 ia.get_movie() 获取电影指定属性信息（30种关系）写入 tsv 文件【完成】
3. 根据前一步得到的结果构建三元组数据集【完成】
4. 参考 WN 等数据集进行数据集划分（用的时候要 shuffle）【完成】
5. 三元组可视化（python request 或 neo4j）（选做）

In [ ]:
# 这部分为测试代码

import time
import imdb
import pandas as pd
from tqdm import tqdm

# help(imdb.Movie.Movie)
ia = imdb.IMDb()
basic_data = pd.read_csv('./title_basic_subset.tsv', sep='\t', header=0, usecols=[0,1], low_memory=False)
movie_items = basic_data[(basic_data['titleType']=='movie')]
movie_ids = movie_items['tconst'].values.tolist()

# 指定的属性标签
relations = ["cast", "directors", "writers", "producers", "composers", "cinematographers", "editors", "editorial department", 
             "casting directors", "production designers", "art directors", "set decorators", "costume designers", "make up department",
             "production managers", "assistant directors", "art department", "sound department", "special effects", "visual effects",
             "stunts", "camera department", "casting department", "costume departmen", "location management", "music department",
             "script department", "transportation department", "production companies", "distributors"]

time_start=time.time()
with open('./movie_triples.tsv', 'w') as f:
    for movie_id in tqdm(movie_ids):
        movie = ia.get_movie(movie_id[2:])
        for relation in relations:
            try:
                pro_values = movie[relation][:5]    # 只取前10个
            except KeyError:
                continue
            else:
                for entity in pro_values:
                    if entity.getID():    # 有的实体可能为空；实体可以为人或公司，存储时要用带前缀的ID，因为名字可能会有非法字符
                        if isinstance(entity, imdb.Person.Person):
                            f.write(movie_id + '\t' + '_'.join(relation.split(' ')) + '\t' + 'nm' + entity.personID + '\n')
                        elif isinstance(entity, imdb.Company.Company):
                            f.write(movie_id + '\t' + '_'.join(relation.split(' ')) + '\t' + 'co' + entity.companyID + '\n')
                        else:    # 实体类型既不是人物，也不是公司
                            continue
time_end=time.time()
print('Cost time: %.2f s.' %(time_end-time_start))

### 数据规模
- 用于试验的小数据集 3.09kb，有 27 条电影数据，生成的三元组文件大小为 38.7kb，三元组 930 条，用时 94.5s
- 正式数据集大小为 704219kb，有 **593631** 条电影数据，全爬的话预估生成的三元组文件大小为 850871kb 即 850M，三元组 20447290 条，预估用时 2121681s 即 589h
- 解决方法：分块爬取，基本 1000 条数据对应 1h，分块爬取，得到约 590 块数据，每块数据 1M 多，约三万多条三元组数据

### 待办
- 先爬一个小数据集做实验，后面有时间了再爬完整的（预估要 20 多天）
- 调研有没有人已经做了 IMDB KG 的工作，我们的优势在哪里

In [1]:
import time
import imdb
import pandas as pd
from tqdm import tqdm

# help(imdb.Movie.Movie)
ia = imdb.IMDb()
basic_data = pd.read_csv('D:/imdb_dataset/title_basic.tsv', sep='\t', header=0, usecols=[0,1], low_memory=False)  # IMDB 原始数据集
movie_items = basic_data[(basic_data['titleType']=='movie')]
movie_ids = movie_items['tconst'].values.tolist()

# 指定的属性标签
relations = ["cast", "directors", "writers", "producers", "composers", "cinematographers", "editors", "editorial department", 
             "casting directors", "production designers", "art directors", "set decorators", "costume designers", "make up department",
             "production managers", "assistant directors", "art department", "sound department", "special effects", "visual effects",
             "stunts", "camera department", "casting department", "costume departmen", "location management", "music department",
             "script department", "transportation department", "production companies", "distributors"]

In [9]:
block_idxes = range(594)           # 注意，最后一组（第 594 个 block）不够1000条
block_idxes = range(560, 594)          # 想爬几组就在这里写几到几

for block_idx in block_idxes:
    
    start = 1000 * block_idx    # 该 block 的起止指针
    end = 1000 + start
    movie_id_1000 = movie_ids[start:start+1000]
    file_name = './IMDB30/block_' + str(start) + '_' + str(end) + '.tsv'
    
    time_start=time.time()
    with open(file_name, 'w') as f:         # 有几个 block 循环就要生成几个文件
        for movie_id in tqdm(movie_id_1000):
            movie = ia.get_movie(movie_id[2:])
            for relation in relations:
                try:
                    pro_values = movie[relation][:5]    # 只取前10个
                except KeyError:
                    continue
                else:
                    for entity in pro_values:
                        if entity.getID():    # 有的实体可能为空；实体可以为人或公司，存储时要用带前缀的ID，因为名字可能会有非法字符
                            if isinstance(entity, imdb.Person.Person):
                                f.write(movie_id + '\t' + '_'.join(relation.split(' ')) + '\t' + 'nm' + entity.personID + '\n')
                            elif isinstance(entity, imdb.Company.Company):
                                f.write(movie_id + '\t' + '_'.join(relation.split(' ')) + '\t' + 'co' + entity.companyID + '\n')
                            else:    # 实体类型既不是人物，也不是公司
                                continue
    time_end=time.time()
    print('Cost time of block %d : %.2f s.' %(block_idx, time_end-time_start))
    # 可优化：URLError

 21%|████████                               | 207/1000 [11:55<42:49,  3.24s/it]2022-01-16 11:46:41,188 CRITICAL [imdbpy] E:\Anaconda3\lib\site-packages\imdb\_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt8129950/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 404: ''>},); kwds: {}
Traceback (most recent call last):
  File "E:\Anaconda3\lib\site-packages\imdb\parser\http\__init__.py", line 220, in retrieve_unicode
    response = uopener.open(url)
  File "E:\Anaconda3\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "E:\Anaconda3\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "E:\Anaconda3\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "E:\Anaconda3\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "E

Cost time of block 560 : 3396.36 s.


 15%|█████▊                                 | 149/1000 [08:28<48:47,  3.44s/it]2022-01-16 12:39:50,318 CRITICAL [imdbpy] E:\Anaconda3\lib\site-packages\imdb\_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt8180486/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 404: ''>},); kwds: {}
Traceback (most recent call last):
  File "E:\Anaconda3\lib\site-packages\imdb\parser\http\__init__.py", line 220, in retrieve_unicode
    response = uopener.open(url)
  File "E:\Anaconda3\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "E:\Anaconda3\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "E:\Anaconda3\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "E:\Anaconda3\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "E

Cost time of block 561 : 3394.49 s.


  0%|                                                 | 0/1000 [00:00<?, ?it/s]

...

2022-01-17 19:13:28,410 CRITICAL [imdbpy] E:\Anaconda3\lib\site-packages\imdb\_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt9828428/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 404: ''>},); kwds: {}
Traceback (most recent call last):
  File "E:\Anaconda3\lib\site-packages\imdb\parser\http\__init__.py", line 220, in retrieve_unicode
    response = uopener.open(url)
  File "E:\Anaconda3\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "E:\Anaconda3\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "E:\Anaconda3\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "E:\Anaconda3\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "E:\Anaconda3\lib\urllib\request.py", line 649, in http_error_default
    raise H

2022-01-17 19:50:43,309 CRITICAL [imdbpy] E:\Anaconda3\lib\site-packages\imdb\_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt9866424/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 404: ''>},); kwds: {}
Traceback (most recent call last):
  File "E:\Anaconda3\lib\site-packages\imdb\parser\http\__init__.py", line 220, in retrieve_unicode
    response = uopener.open(url)
  File "E:\Anaconda3\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "E:\Anaconda3\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "E:\Anaconda3\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "E:\Anaconda3\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "E:\Anaconda3\lib\urllib\request.py", line 649, in http_error_default
    raise

Cost time of block 592 : 3315.62 s.


100%|████████████████████████████████████████| 631/631 [34:54<00:00,  3.32s/it]

Cost time of block 593 : 2094.30 s.


---

- 正式数据集需要的文件：entity2id.txt、relation2id.txt、train2id.txt、valid2id.txt、test2id.txt，（type_constrain.txt不知道需不需要）

### 正式数据集各文件构建步骤：

1. 把几块数据整合成一个大的，shuffle，按照 8:1:1 的比例划分 train、valid、test
2. 统计实体和关系数（可以参考早期实现 TransE 的外围代码，看看是用训练集还是全体构建实体/关系字典），指定id，写成 entity2id.txt、relation2id.txt
3. 根据 entity2id.txt、relation2id.txt 将原始的 train、valid、test 转换为 id 表示形式的

### 读原始数据

In [ ]:
import os
import pandas as pd

all_triples = pd.DataFrame(columns=['head', 'relation', 'tail']) 
files = os.listdir('./IMDB30/subset_0_10000/')

for file in files:
    block_triples = pd.read_csv('./IMDB30/subset_0_10000/' + file, sep='\t', header=None, names=['head', 'relation', 'tail'])
    all_triples = pd.concat([all_triples, block_triples], ignore_index=True)

all_triples = all_triples.dropna().applymap(lambda x: str(x).strip())    # 去除空值
all_triples = all_triples.sample(frac=1).reset_index(drop=True)          # shuffle

### 构建实体/关系字典 entity2id.txt、relation2id.txt
- 该数据集可能存在的问题：头实体一定是电影，尾实体一定是人或公司，头尾实体可能不平衡

In [ ]:
from collections import Counter

head_count = Counter(all_triples['head'])
relation_count = Counter(all_triples['relation'])
tail_count = Counter(all_triples['tail'])

# 用于写入文件 entity2id.txt 和 relation2id.txt
# 同时构造 head_dic，tail_dic 和 relation_dic

# entity_list = list((head_count + tail_count).keys())
head_list = list(head_count.keys())
relation_list = list(relation_count.keys())
tail_list = list(tail_count.keys())

head_dic, relation_dic, tail_dic = {}, {}, {}

with open('./IMDB_benchmark/entity2id.txt', 'w') as f:
    f.write(str(len(head_list) + len(tail_list)) + '\n')
    for idx, item in enumerate(head_list):
        f.write(item + '\t' + str(idx) + '\n')
        head_dic[item] = idx
    for idx, item in enumerate(tail_list):
        f.write(item + '\t' + str(idx) + '\n')
        tail_dic[item] = idx
        
with open('./IMDB_benchmark/relation2id.txt', 'w') as f:
    f.write(str(len(relation_list)) + '\n')
    for idx, item in enumerate(relation_list):
        f.write(item + '\t' + str(idx) + '\n')
        relation_dic[item] = idx

### 构建训练集、验证集、测试集 train2id.txt、valid2id.txt、test2id.txt

In [ ]:
# 将三元组的原始字符经过查字典变为 id 表示
id_triples = all_triples.loc[:, ['head', 'tail', 'relation']]    # 存入数据集的顺序是 head, tail, relaiton

id_triples['head'] = id_triples['head'].apply(lambda x: head_dic[x])
id_triples['relation'] = id_triples['relation'].apply(lambda x: relation_dic[x])
id_triples['tail'] = id_triples['tail'].apply(lambda x: tail_dic[x])

id_triples = id_triples.sample(frac=1).reset_index(drop=True)     # 再 shuffle 一次

In [ ]:
# 划分数据集

triple_count = all_triples.shape[0]

train_number = int(triple_count * 0.8)
valid_number = (triple_count - train_number) // 2
test_number = triple_count - train_number - valid_number

train_set = id_triples.iloc[:train_number]
valid_set = id_triples.iloc[train_number : train_number + valid_number]
test_set = id_triples.iloc[train_number + valid_number:]

print(train_number, valid_number, test_number)

In [ ]:
# 数据集 dump 进文件：第一行是数量

with open('./IMDB_benchmark/train2id.txt', 'w') as f:
    f.write(str(train_number) + '\n')
    
with open('./IMDB_benchmark/valid2id.txt', 'w') as f:
    f.write(str(valid_number) + '\n')
    
with open('./IMDB_benchmark/test2id.txt', 'w') as f:
    f.write(str(test_number) + '\n')

train_set.to_csv('./IMDB_benchmark/train2id.txt', sep=' ', header=False, index=False, mode='a')    # 文件是否要事先建好？csv能否 dump txt？
valid_set.to_csv('./IMDB_benchmark/valid2id.txt', sep=' ', header=False, index=False, mode='a')
test_set.to_csv('./IMDB_benchmark/test2id.txt', sep=' ', header=False, index=False, mode='a')